# LangChain Agent

From https://python.langchain.com/v0.1/docs/modules/agents/how_to/custom_agent/

In [1]:
import os

from dotenv import load_dotenv

In [10]:
load_dotenv()
from langchain_openai import AzureChatOpenAI
from langchain.agents import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor

## Set up LLM

In [3]:
llm = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
)

## Define tools

In [4]:
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

In [5]:
# get_word_length.invoke("abc")

In [6]:
tools = [get_word_length]

## Agent instruction

> Because OpenAI Function Calling is finetuned for tool usage, we hardly need any instructions on how to reason, or how to output format. We will just have two input variables: input and agent_scratchpad. input should be a string containing the user objective. agent_scratchpad should be a sequence of messages that contains the previous agent tool invocations and the corresponding tool outputs.

In [7]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

## Create agent

In [8]:
# Bind tools
llm_with_tools = llm.bind_tools(tools)

In [9]:
# Create agent
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [13]:
# Create executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

## Invoke

In [14]:
list(agent_executor.stream({"input": "How many letters in the word eudca"}))



> Entering new AgentExecutor chain...
Could not parse tool input: {'arguments': "const word = 'eudca';\nfunctions.get_word_length({word: word});", 'name': 'python'} because the `arguments` is not valid JSON.Invalid or incomplete responseHere's a function that will return the length of a given word:

```
functions.get_word_length = ({ word }) => {
  return word.length;
};
```

Using this function, the word "eudca" has 5 letters.

> Finished chain.


[{'steps': [AgentStep(action=AgentAction(tool='_Exception', tool_input='Invalid or incomplete response', log='Could not parse tool input: {\'arguments\': "const word = \'eudca\';\\nfunctions.get_word_length({word: word});", \'name\': \'python\'} because the `arguments` is not valid JSON.'), observation='Invalid or incomplete response')],
  'messages': [HumanMessage(content='Invalid or incomplete response')]},
 {'output': 'Here\'s a function that will return the length of a given word:\n\n```\nfunctions.get_word_length = ({ word }) => {\n  return word.length;\n};\n```\n\nUsing this function, the word "eudca" has 5 letters.',
  'messages': [AIMessage(content='Here\'s a function that will return the length of a given word:\n\n```\nfunctions.get_word_length = ({ word }) => {\n  return word.length;\n};\n```\n\nUsing this function, the word "eudca" has 5 letters.')]}]